# <center>Capstone Project</center>

#### This notebook will be used for the final project in the specialization. It will be updated and description will be changed from time to time

In [1]:
import pandas as pd
import numpy as np

In [2]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


In [3]:
import requests
from bs4 import BeautifulSoup
import csv
import json
import xml
import pandas as pd
import numpy as np

!conda install -c conda-forge folium=0.5.0 --yes
import folium
from folium import plugins

import json
from pprint import pprint

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\ProgramData\Anaconda3

  added / updated specs:
    - geopy


The following packages will be UPDATED:

  geopy                                         1.19.0-py_0 --> 1.20.0-py_0


Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done


In [ ]:
url = requests.get('https://en.wikipedia.org/wiki/Arrondissements_of_Paris#Arrondissements').text

#### Apply Beautiful Soup to make Soup of the URL, then apply Prettify to view the nested tags and look for the tag that contains the table with arrondissement information

In [ ]:
soup = BeautifulSoup(url, 'lxml')
print(soup.prettify())

In [ ]:
arrond_table = soup.find("table", class_ = 'wikitable sortable')
arrond_table_rows = arrond_table.find_all('tr')

##### Extract the columns Arrondissement, Name, Area, Population, Density

In [ ]:
information = []
for row in arrond_table_rows:
    info = row.text.split('\n')[1:-1]
    information.append(info)

del information[0][1::2] # There are extra column headers "" that we don't need, so delete them    

information

#### Turn above information into a Pandas dataframe

In [ ]:
arrond_df = pd.DataFrame(information[1:], columns=information[0])
# where information[1:] contains each row of neighborhoods
# and columns = information[0] gives the column names

arrond_df.head(25)

In [ ]:
# Add a column with just the Arrondissement numbers

arrond_number = ['1st', '2nd', '3rd', '4th', '5th', '6th', '7th', '8th', '9th','10th',
                '11th', '12th', '13th', '14th', '15th', '16th', '17th', '18th', '19th', '20th']

Arr = pd.DataFrame({'Arrondissement': arrond_number})

arrond_df = pd.concat([arrond_df, Arr], axis = 1)

arrond_df.head()

In [ ]:
arrond_df.dtypes

#### Here's a dataframe of the top Paris architectural and historical landmarks I've gathered manually from the Architecture Digest website

In [ ]:
landmarks = ['Cathédrale Notre-Dame de Paris', 'Arc de Triomphe', 'Centre Pompidou',
             'Sacré-Cœur Basilica', 'Eiffel Tower',
             'The Louvre', 'La Grande Arche', 'Les Invalides', 'Fondation Louis Vuitton',
             'Panthéon', 'Picasso Museum', 'Cinémathèque Française',
             'Opéra Garnier', 'The Grand Palais', 'Hôtel de Ville',
             'Fondation Cartier', 'Jeu de Paume', 'Jardin des Plantes',
             'Fondation Jerôme Seydoux-Pathé', 'Île de la Cité',
             'The Palais de Tokyo', 'Philharmonie de Paris', 'Musée d’Orsay',
             'Palais du Luxembourg', 'Montmartre', 'Champs-Élysées',
             'Palace of Versailles', 'Tuileries Garden', 'Moulin Rouge',
             'Disneyland Paris', 'Place de la Concorde', 'Catacombs of Paris',
             'Rodin Museum', 'Île de la Cité', 'Orangerie Museum', 'Pont des Arts',
             'Île Saint-Louis', 'Place Vendôme', 'Saint-Jacques Tower', 'La Conciergerie',
            ]

landmarks_df = pd.DataFrame({'Landmark': landmarks})

#### I will find each landmark's latitude and longitude information using the Nominatim package.

In [ ]:

def landmark_loc(list_of_landmarks):
    for place in list_of_landmarks:
        address = (place)
        geolocator = Nominatim(user_agent="Paris_explorer")
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
        print('{''}, {}, {},'.format(place,latitude,longitude))

landmark_loc(landmarks)

#### I will turn the above information into a dataframe (need to find a way to do this more efficiently)

In [ ]:
landmarks_loc = ['Cathédrale Notre-Dame de Paris', 48.85293695, 2.35005149954546,
'Arc de Triomphe', 48.8737791, 2.29503722603767,
'Centre Pompidou', 48.8605918, 2.35247431766534,
'Sacré-Cœur Basilica', 48.88680575, 2.34301534488351,
'Eiffel Tower', 48.8582602, 2.29449905431968,
'The Louvre', 48.8611473, 2.33802768704666,
'La Grande Arche', 48.8930151, 2.2376206,
'Les Invalides', 48.8570374, 2.3118779,
'Fondation Louis Vuitton', 48.87665005, 2.26333809580246,
'Panthéon', 48.8454189, 2.35258442491778,
'Picasso Museum', 41.3851066, 2.18120508674285,
'Cinémathèque Française', 48.8369987, 2.38263520273537,
'Opéra Garnier', 48.87203085, 2.3317901855896,
'The Grand Palais', 48.86616135, 2.31222295966943,
'Hôtel de Ville', 43.9492974, 4.80526012041732,
'Fondation Cartier', 48.8373182, 2.33193184219906,
'Jeu de Paume', 48.8658156, 2.32408776132074,
'Jardin des Plantes', 48.8432224, 2.35950895709484,
'Fondation Jerôme Seydoux-Pathé', 48.8334765, 2.35490681339522,
'Île de la Cité', 48.85470505, 2.3474846899114,
'The Palais de Tokyo', 48.8639905, 2.29652151313114,
'Philharmonie de Paris', 48.89192125, 2.39413864295307,
'Musée d’Orsay', 48.85991785, 2.32658283469656,
'Palais du Luxembourg', 48.84852845, 2.33683647732996,
'Montmartre', 46.8021347, -0.7198939,
'Champs-Élysées', 48.8707573, 2.3053312,
'Palace of Versailles', 48.80442525, 2.1202852709926,
'Tuileries Garden', 48.86366215, 2.32683861305756,
'Moulin Rouge', 48.8840787, 2.3324082,
'Disneyland Paris', 48.8711359, 2.77612651196751,
'Place de la Concorde', 29.9002052, -90.0095146,
'Catacombs of Paris', 48.833964, 2.3324924,
'Rodin Museum', 39.9619295, -75.1739508,
'Île de la Cité', 48.85470505, 2.3474846899114,
'Orangerie Museum', 48.8637655, 2.32266020780693,
'Pont des Arts', 48.8578114, 2.3372459,
'Île Saint-Louis', 48.85161305, 2.35673414953666,
'Place Vendôme', 48.8674634, 2.32942811682519,
'Saint-Jacques Tower', 48.8580032, 2.34892171290132,
'La Conciergerie', 48.8559492, 2.3460263]

landmarks_df = pd.DataFrame(np.array(landmarks_loc).reshape(40, 3), columns = ["Landmark","Latitude","Longitude"])

# Change the latitude and longitude to a float
landmarks_df['Latitude'] = landmarks_df.Latitude.astype(float)
landmarks_df['Longitude'] = landmarks_df.Longitude.astype(float)

#### Get latitude/longitude for each arrondissement

In [ ]:

# turn numbered arrondissements into a list
arrond_list = arrond_df['Arrondissement'].unique()
arrond_list

def landmark_loc(list_of_arronds):
    for place in list_of_arronds:
        address = (place + ' Arrondissement')
        geolocator = Nominatim(user_agent="arrond_explorer")
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
        print('{}, {}, {},'.format(place,latitude,longitude))

landmark_loc(arrond_list)

In [ ]:
# turn numbered arrondissements into a list
arrond_list = arrond_df['Arrondissement'].unique()
arrond_list

def landmark_loc(list_of_arronds):
    LL = []
    for place in list_of_arronds:
        address = (place + ' Arrondissement')
        geolocator = Nominatim(user_agent="arrond_explorer")
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
        LL.append(place)
        LL.append(latitude)
        LL.append(longitude)
    print(LL)


LL = landmark_loc(arrond_list)

In [ ]:

# not sure why the LL above couldn't be reshaped, so copied them below to insert into a dataframe converter

LL = ['1st', 48.8646144, 2.334396, '2nd', 48.868743, 2.341688, '3rd', 48.864212, 2.360936, 
      '4th', 48.8562021, 2.3556193, '5th', 48.8460591, 2.3445228, '6th', 48.8504333, 2.3329507, 
      '7th', 48.8570281, 2.3201953, '8th', 48.8773158, 2.3172611, '9th', 48.876019, 2.339962, 
      '10th', 48.876106, 2.35991, '11th', 48.858416, 2.379703, '12th', 48.8396154, 2.3957517, 
      '13th', 48.8323973, 2.3555829, '14th', 48.8330589, 2.3269591, '15th', 48.8413705, 2.3003827, 
      '16th', 48.8631709, 2.2757648, '17th', 48.8842239, 2.3223639, '18th', 48.8900117, 2.3464668, 
      '19th', 48.889343, 2.38436, '20th', 48.865042, 2.398929]

In [ ]:
arrond_LL = pd.DataFrame(np.array(LL).reshape(20,3), columns = ["Arrondissement","Latitude","Longitude"])

# Change the latitude and longitude to a float
arrond_LL['Latitude'] = arrond_LL.Latitude.astype(float)
arrond_LL['Longitude'] = arrond_LL.Longitude.astype(float)

arrond_LL

In [ ]:
Next, I will upload the GEO JSON file of Paris and attempt to create a choroplath map

In [ ]:
with open('arrondissements_osm_paris.geojson') as file:    
    paris = json.load(file)
    
paris

#### Exploration of the data

In [ ]:
address = ('Paris, France')
geolocator = Nominatim(user_agent="Paris_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('{''}, {}, {},'.format(address,latitude,longitude))

In [ ]:
paris_lat = 48.8566101
paris_long = 2.3514992

paris_map = folium.Map(location=[paris_lat, paris_long], zoom_start = 12)
paris_map

#### Generate a map that show the borders of the arrondissement and labels

In [ ]:
arronds_map = folium.Map(location = [paris_lat, paris_long], zoom_start = 14)

arronds_map.choropleth(
    geo_data=paris,
    key_on='feature.properties.nom',
    fill_color='PuBu', 
    fill_opacity=0.1, 
    line_opacity=0.8,
)

#add neighborhood markers to the Paris map
for lat, long, arrond in zip(arrond_LL['Latitude'], arrond_LL['Longitude'], 
                                 arrond_LL['Arrondissement']):
    label = '{}'.format(arrond)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius = 7, 
        popup = label,
        color = 'red',
        fill = True,
        fill_color = 'blue',
        fill_opacity = 0.7,
        parse_html = False).add_to(arronds_map)
        
# display map
arronds_map

#### Generate choropleth map using the geo json file and the population information

In [ ]:
# make new list so that "nom" in geojson matches the arrond in the data

arrond_new = ['1er Arrondissement', '2e Arrondissement','3e Arrondissement',
             '4e Arrondissement', '5e Arrondissement', '6e Arrondissement', 
             '7e Arrondissement', '8e Arrondissement','9e Arrondissement',
             '10e Arrondissement', '11e Arrondissement', '12e Arrondissement', 
             '13e Arrondissement', '14e Arrondissement', '15e Arrondissement',
             '16e Arrondissement', '17e Arrondissement', '18e Arrondissement', 
             '19e Arrondissement', '20e Arrondissement']

# this is the population list. Was going to take this from the dataframe, but it had commas and was an object.o
population_2005 = [17700, 20700, 35100, 28600, 60600,
45200,55400,38700,58500,88800,152500,138300,181300,134700,
232400,149500,160300,188700,187200,191800]


pop_2005 = pd.DataFrame({'Arrondissement': arrond_new, 'Population': population_2005})

#### Here is a choropleth map of Paris, where darker shades signify a higher Parisan population

In [ ]:
Paris_map = folium.Map(location = [paris_lat, paris_long], zoom_start = 14)

Paris_map.choropleth(
    geo_data=paris,
    data=pop_2005,
    columns=['Arrondissement', 'Population'],
    key_on='feature.properties.nom',
    fill_color='Reds', 
    fill_opacity=0.5, 
    line_opacity=0.2,
    legend_name='Population by Arrondissement in Paris 2005'
)


# display map
Paris_map

#### Map of Parisian arrondissement (labeled) along with population information

In [ ]:
paris_map_v2 = folium.Map(location = [paris_lat, paris_long], zoom_start = 12)

paris_map_v2.choropleth(
    geo_data=paris,
    data=pop_2005,
    columns=['Arrondissement', 'Population'],
    key_on='feature.properties.nom',
    fill_color='PuBu', 
    fill_opacity=0.5, 
    line_opacity=0.2,
    legend_name='Population by Arrondissement in Paris 2005'
)

#add neighborhood markers to the Paris map
for lat, long, arrond in zip(arrond_LL['Latitude'], arrond_LL['Longitude'], 
                                 arrond_LL['Arrondissement']):
    label = '{}'.format(arrond)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius = 7, 
        popup = label,
        color = 'red',
        fill = True,
        fill_color = 'blue',
        fill_opacity = 0.7,
        parse_html = False).add_to(paris_map_v2)
        

# display map
paris_map_v2

#### Another option for a choroplath map with arrond long/lat points

In [ ]:
from folium import IFrame

paris_map_v3 = folium.Map(location = [paris_lat, paris_long], zoom_start = 12)

paris_map_v3.choropleth(
    geo_data=paris,
    data=pop_2005,
    columns=['Arrondissement', 'Population'],
    key_on='feature.properties.nom',
    fill_color='PuBu', 
    fill_opacity=0.5, 
    line_opacity=0.2,
    legend_name='Population by Arrondissement in Paris 2005'
)

#add neighborhood markers to the Paris map
for lat, long, arrond in zip(arrond_LL['Latitude'], arrond_LL['Longitude'], 
                                 arrond_LL['Arrondissement']):
    label = '{}'.format(arrond)
    iframe = folium.IFrame(label, width=700, height=450)
    label = folium.Popup(iframe, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius = 7, 
        color = 'red',
        fill = True,
        fill_color = 'blue',
        fill_opacity = 0.7,
        parse_html = False).add_to(paris_map_v3)
        

# display map
paris_map_v3

#### Map of Paris with markers of top tourists landmarks

In [ ]:

# create a map of Paris
map_landmarks = folium.Map(location = [paris_lat, paris_long], zoom_start = 10)

map_landmarks.choropleth(
    geo_data=paris,
    key_on='feature.properties.nom',
    fill_color='PuBu', 
    fill_opacity=0.1, 
    line_opacity=0.8,
)

#add neighborhood markers to the Paris map
for lat, long, landmark in zip(landmarks_df['Latitude'], landmarks_df['Longitude'], 
                                 landmarks_df['Landmark']):
    label = '{}'.format(landmark)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius = 7, 
        popup = label,
        color = 'red',
        fill = True,
        fill_color = 'blue',
        fill_opacity = 0.7,
        parse_html = False).add_to(map_landmarks)
        
map_landmarks

#### Map of Paris with landmarks and arrondissements

In [ ]:
# create a map of Paris
map_landmarks2 = folium.Map(location = [paris_lat, paris_long], zoom_start = 10)

map_landmarks2.choropleth(
    geo_data=paris,
    data=pop_2005,
    columns=['Arrondissement', 'Population'],
    key_on='feature.properties.nom',
    fill_color='PuBu', 
    fill_opacity=0.1, 
    line_opacity=0.5,
)

#add landmarks markers to the Paris map
for lat, long, landmark in zip(landmarks_df['Latitude'], landmarks_df['Longitude'], 
                                 landmarks_df['Landmark']):
    label = '{}'.format(landmark)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius = 7, 
        popup = label,
        color = 'red',
        fill = True,
        fill_color = 'blue',
        fill_opacity = 0.7,
        parse_html = False).add_to(map_landmarks2)
    
#add neighborhood markers to the Paris map
for lat, long, arrond in zip(arrond_LL['Latitude'], arrond_LL['Longitude'], 
                                 arrond_LL['Arrondissement']):
    label = '{}'.format(arrond)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius = 5, 
        popup = label,
        color = 'white',
        fill = True,
        fill_color = 'green',
        fill_opacity = 0.5,
        parse_html = False).add_to(map_landmarks2)
        
map_landmarks2

#### Let's bring in the Foursquare data

In [ ]:
CLIENT_ID = 'EYOIMD2CUHSNE3YB3XQJI2UNLSL4M35Q12YTC2DVRCGSKGAD' # your Foursquare ID
CLIENT_SECRET = 'DFMF3LXGTQ5NNG1W25RIJPRZIQSAZUABHRJKTOGDLUOLUQWH' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

#### Now, let's get the top 250 venues from each of the arrondissement with the most landmarks, the 1st, 4th, 7th, and 8th. From online research, other vistor favorites are the 5th, 18th, 6th, and 3rd arrondissements.

In [ ]:
# Make dataframe of the most popular arrondissement


popular_arrond = arrond_LL[arrond_LL['Arrondissement'].isin(
    ['1st', '3rd', '4th', '5th', '6th', '7th', '8th', '18th'])]

popular_arrond

In [ ]:
# Example call of 1st arrondissement

arrond_1_lat = arrond_LL.iloc[0,1]

arrond_1_long = arrond_LL.iloc[0,2]

LIMIT = 250

radius = 2000



url1 = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    arrond_1_lat, 
    arrond_1_long, 
    VERSION, 
    radius, 
    LIMIT)

url1

#### Send the GET request and examine the results

In [ ]:
results1 = requests.get(url1).json()
results1